# Chains in LangChain

## Outline
* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [ ]:
import datetime

current_date = datetime.datetime.now().date()

target_date = datetime.date(2024, 6, 12)

if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [ ]:
# Import pandas
import pandas as pd

In [ ]:
# Load dataset
df = pd.read_csv('l3_data.csv')
df.head()

## LLMChain

In [ ]:
from langchain_openai import ChatOpenAI  # The OpenAI model
from langchain.prompts import ChatPromptTemplate  # The prompt
from langchain.chains import LLMChain  # The llm chain

In [ ]:
# Initialize the language model
llm = ChatOpenAI(temperature=0.9)

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)

In [ ]:
# Combine the model and the prompt into a chain
chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
product = "Queen size Sheet Set"
# chain.run(product)  # Deprecated use `invoke` instead of `run`
chain.invoke(product)